# Data Analysis

## Solutions object
Obtain the Hidden Sector from: https://zenodo.org/record/5526707#.YzwIANJBzeQ and add the missing the column `Hidden Sector` of the following data scheme

![img](./squeme.svg)

### Example of entry

In [1]:
from IPython.display import JSON 
import pandas as pd

In [2]:
JSON({'A':1})

<IPython.core.display.JSON object>

In [3]:
JSON(([{'l':[1,3],'k':[2],
       'solution':[1,2,-3,4,5],'gcd':1,'n':5,
      'hidden':[{'ψ':[(1,5),(2,4),(-3,)],
                 'S':6}
               ]
             }] 
            )
    )


<IPython.core.display.JSON object>

In [4]:
import pandas as pd

In [5]:
pd.DataFrame([{'l':[1,3],'k':[2],
       'solution':[1,2,-3,4,5],'gcd':1,'n':5,
      'hidden':[{'ψ':[(1,5),(2,4),(-3,)],
                 'S':6}
               ]
             }] 
            )

,l,k,solution,gcd,n,hidden
0,"[1, 3]",[2],"[1, 2, -3, 4, 5]",1,5,"[{'ψ': [(1, 5), (2, 4), (-3,)], 'S': 6}]"


### Read the full dataset

In [6]:
df=pd.read_json('https://zenodo.org/record/5526707/files/solutions.json?download=1')

In [7]:
df['zmax']=df['solution'].str[-1].abs()
df=df.sort_values(['n','zmax']).reset_index(drop=True)

#How many solutions with n=12 and z_max <= 20?
df[ (df['n']==12) & (df['zmax']<=20) ].shape

(16862, 6)

### Add the new column with the list of hidden sectors

In [8]:
import itertools

In [9]:
list(itertools.combinations_with_replacement([1,2],2))

[(1, 1), (1, 2), (2, 2)]

In [61]:
import math as mh
def get_hidden_sector(l):
    '''
    Extract massive pairs without massles fermions from l
    '''
    sltns=[]
    ps = list(set(itertools.combinations_with_replacement(l,2)))
    Ss = set(abs(sum(x)) for x in ps)
    # capture all the possible scalar, Ss, values
    for s in set(Ss):
        msv = [p for p in ps if abs(sum(p))==s]
        #SAFE Discard uneven Dirac pairs when the greater count is not Majorana
        if [(x,y) for x,y in msv if x!=y and 
                  (l.count(x)<l.count(y) and abs(sum((y,y)))!=s or
                   l.count(x)>l.count(y) and abs(sum((x,x)))!=s
            )]:            
            continue
            
        #msv=list(set(msv))
        # flatten the list
        massives=set([x for sublist in msv for x in sublist])
        massless=set(l).difference(massives)
        if not massless:
            sltn={}
            sltn['S']=s
            #Save storage
            sltn['ψ']=msv
            sltns.append(sltn)
            
    return sltns

assert get_hidden_sector([1,2,-3,4,5])[0].get('S')==6
assert get_hidden_sector([1,2,-3,-3,4,5])[0].get('S')==6
assert get_hidden_sector([1,1,2,-3,-3,4,5,5])[0].get('S')==6
#aψ1ψ1+bψ1ψ2+cψ1,ψ3 → https://www.wolframalpha.com/input?i=Rank%20{{a,b,c},{b,0,0},{c,0,0}}
assert get_hidden_sector([1,2,-3,4,5,9,9])==[]
get_hidden_sector([1,1,2,-3,4,5])
#aψ1ψ2+bψ1ψ3 → https://www.wolframalpha.com/input?i=rank+{{0,a,b},{a,0,0},{b,0,0}}
assert get_hidden_sector([1,1,2,-3,4,5])==[]
assert get_hidden_sector([1,1,2,-3,-3,4,5])==[]
assert get_hidden_sector([1,2,-3,4,5,8])==[]
assert get_hidden_sector([1,2,-3,4,5,8,8])==[]
assert get_hidden_sector([])==[]
assert get_hidden_sector([1, 1, 1, 1, 1, -2, -2, -2, -2, 3])==[]
assert get_hidden_sector([1, 2, 2, 2, -3, -5, -6, 7])[0].get('S')==4
assert get_hidden_sector( [1, 2, 2, 4, -5, -5, -7, 8] )==[]

In [ ]:
%%time
df['hidden']=df['solution'].apply(get_hidden_sector)

In [ ]:
hs=df[df['hidden'].apply(len)>0].reset_index(drop=True)#.iloc[0].to_dict()

In [ ]:
hs.shape

## Ask question to the solutions object

* Test the solution
* Which is the minimum hidden sector and with the least charge in absolute value
* Check for multigenerational solution
* Check for unconditional stability
* Solutions with active symmetry
* How many solutions with n=12 and zmax<=20
* Check for Majorana and Dirac fermions

In [15]:
pd.set_option('display.max_colwidth',200)

In [16]:
# What is the minimum n?
hs['n'].min()

7

In [17]:
# What is the minimum z_max?
hs['zmax'].min()

6

In [18]:
hs[hs['zmax']==hs['zmax'].min()]

,l,k,solution,gcd,n,zmax,hidden
1739,"[-1, 0, -1, 0, 1]","[-3, -2, -3, -2, 2]","[1, -2, -2, -2, 3, 3, 4, 4, -5, -5, -5, 6]",1,12,6,"[{'S': 7, 'ψ': [(1, 6), (-2, -5), (3, 4)]}]"


In [19]:
hs[:3]

,l,k,solution,gcd,n,zmax,hidden
0,"[-2, 0]","[-1, -2, -3]","[3, 5, -8, 9, -10, -14, 15]",1,7,15,"[{'S': 5, 'ψ': [(9, -14), (-10, 15), (3, -8), (5, -10)]}]"
1,"[2, 0]","[-1, -2, -3]","[5, 6, 7, -10, -11, -12, 15]",1,7,15,"[{'S': 5, 'ψ': [(7, -12), (-10, 15), (5, -10), (6, -11)]}]"
2,"[-1, 2, -2]","[-7, 4, 0]","[1, 2, 2, 2, -3, -5, -6, 7]",1,8,7,"[{'S': 4, 'ψ': [(1, -5), (-3, 7), (2, -6), (2, 2)]}]"


In [20]:
# How many solutions have several dark sectors?
hs[hs['hidden'].apply(len)>1].shape

(494, 7)

In [21]:
hs[hs['hidden'].apply(len)>2].shape

(0, 7)

In [22]:
# What is the minimum number of DM particles?
hs['hidden'].str[0].str['ψ'].apply(len).min()

3

In [23]:
hs[hs['hidden'].str[0].str['ψ'].apply(len)==hs['hidden'].str[0].str['ψ'].apply(len).min()][:1]

,l,k,solution,gcd,n,zmax,hidden
3,"[-1, 0, 1]","[-4, -2, 2]","[1, -2, -2, 4, 5, -7, -7, 8]",1,8,8,"[{'S': 9, 'ψ': [(4, 5), (1, 8), (-2, -7)]}]"


### Same questions for SM solutions

In [24]:
df3=df[df['solution'].apply(lambda l: 3 in [l.count(x) for x in l])].reset_index(drop=True)

In [25]:
def get_dark(l,idx=0):
    #TODO,
    xxx=list(set([x for x in l if l.count(x)==3]))
    if len(xxx)>=idx+1:
        xx=xxx[idx]
        return [x for x in l if x!=xx]
    else:
        return []
assert get_dark([1,1,2,2,2,3] )==[1, 1, 3]
assert get_dark([2,2,2,2,-5,-5,-5,7])==[2, 2, 2, 2, 7]

In [26]:
df3['hidden']=df3['solution'].apply(get_dark).apply(get_hidden_sector)

In [27]:
hs3=df3[df3['hidden'].apply(len)>0].reset_index(drop=True)
hs3.shape

(117, 7)

In [28]:
# idxmax==1
df3['hidden']=df3['solution'].apply(lambda l: get_dark(l,idx=1)).apply(get_hidden_sector)
hs3=pd.concat((hs3,df3[df3['hidden'].apply(len)>0])).reset_index(drop=True)
hs3.shape

(121, 7)

In [29]:
# idxmax==1
df3['hidden']=df3['solution'].apply(lambda l: get_dark(l,idx=2)).apply(get_hidden_sector)
hs3=pd.concat((hs3,df3[df3['hidden'].apply(len)>0])).reset_index(drop=True)
hs3.shape

(121, 7)

In [30]:
hs3['zmax'].min()

9

In [31]:
hs3[:1]

,l,k,solution,gcd,n,zmax,hidden
0,"[1, 0, -2]","[-3, -1, 1]","[3, 3, 3, 5, -7, -9, -11, 13]",2,8,13,"[{'S': 18, 'ψ': [(-7, -11), (-9, -9), (5, 13)]}]"


In [32]:
hs3[hs3['hidden'].apply(len)>1].shape

(4, 7)

In [33]:
hs3[hs3['hidden'].apply(len)>2].shape

(2, 7)

In [34]:
hs3[hs3['hidden'].apply(len)>3].shape

(0, 7)

In [35]:
hs3['hidden'].str[0].str['ψ'].apply(len).min()

2

In [36]:
hs3[hs3['hidden'].str[0].str['ψ'].apply(len)==hs3['hidden'].str[0].str['ψ'].apply(len).min()]

,l,k,solution,gcd,n,zmax,hidden
4,"[-2, 0, 2]","[-1, 1, 0, -1]","[1, 1, -4, -5, 9, 9, 9, -10, -10]",1,9,10,"[{'S': 9, 'ψ': [(1, -10), (-4, -5)]}]"
6,"[-1, 0, 1]","[-1, -3, 0, -1]","[1, 1, -3, -8, 11, 11, 11, -12, -12]",1,9,12,"[{'S': 11, 'ψ': [(-3, -8), (1, -12)]}]"
8,"[-1, 0, -1, 1]","[-3, 0, -1, 0]","[4, -5, -5, -6, -6, 11, 11, 11, -15]",1,9,15,"[{'S': 11, 'ψ': [(4, -15), (-5, -6)]}]"
16,"[-1, 0, 1]","[-1, -5, 0, 5]","[2, 2, -3, 15, 15, -17, -17, -17, 20]",3,9,20,"[{'S': 17, 'ψ': [(2, 15), (-3, 20)]}]"
18,"[-1, 0, -1, 1]","[-3, 0, -3, 0]","[2, 2, -7, -12, 19, 19, 19, -21, -21]",1,9,21,"[{'S': 19, 'ψ': [(-7, -12), (2, -21)]}]"
19,"[-1, 0, 1]","[-1, 1, 0, -7]","[3, 3, -4, 14, 14, -17, -17, -17, 21]",3,9,21,"[{'S': 17, 'ψ': [(-4, 21), (3, 14)]}]"
24,"[1, 0, 1, -1]","[-3, 0, -3, 0]","[4, 4, -5, 15, 15, -19, -19, -19, 24]",1,9,24,"[{'S': 19, 'ψ': [(4, 15), (-5, 24)]}]"


### Extend the data scheme with number of generations

In [37]:
def get_Dirac_and_Majorana_generations(l,h):
    g=[]
    dgmax=0
    mgmax=0
    for idx in range(len(h)):
        for x,y in h[idx].get('ψ'):
            if x!=y and l.count(x)>dgmax and l.count(y)==l.count(x):
                    dgmax=l.count(x)
            if x==y and l.count(x)>mgmax:
                    mgmax=l.count(x)
        g.append({'DG':dgmax,'MG':mgmax})
    return g
l=[2, 2, -3, 15, 15, -17, -17, -17, 20]
h=[{'S': 17, 'ψ': [(2, 15), (20, -3)]}]
assert get_Dirac_and_Majorana_generations(l,h)==[{'DG': 2, 'MG': 0}]

In [38]:
hs['Generations']=hs.apply(lambda row: get_Dirac_and_Majorana_generations(row['solution'],row['hidden']),axis='columns')

In [39]:
hs3['Generations']=hs3.apply(lambda row: get_Dirac_and_Majorana_generations(row['solution'],row['hidden']),axis='columns')

In [40]:
hs['Gmax']=-hs['Generations'].str[0].apply(lambda d: sum(d.values()))

In [41]:
hs=hs.sort_values(['n','Gmax']).reset_index(drop=True)

In [42]:
hs[hs['n']==7]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
0,"[-2, 0]","[-1, -2, -3]","[3, 5, -8, 9, -10, -14, 15]",1,7,15,"[{'S': 5, 'ψ': [(9, -14), (-10, 15), (3, -8), (5, -10)]}]","[{'DG': 1, 'MG': 0}]",-1
1,"[2, 0]","[-1, -2, -3]","[5, 6, 7, -10, -11, -12, 15]",1,7,15,"[{'S': 5, 'ψ': [(7, -12), (-10, 15), (5, -10), (6, -11)]}]","[{'DG': 1, 'MG': 0}]",-1


In [43]:
hs[hs['n']==8][:6]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
2,"[-1, 2, -2]","[-7, 4, 0]","[1, 2, 2, 2, -3, -5, -6, 7]",1,8,7,"[{'S': 4, 'ψ': [(1, -5), (-3, 7), (2, -6), (2, 2)]}]","[{'DG': 1, 'MG': 3}]",-4
3,"[1, 0, -2]","[-3, -1, 1]","[3, 3, 3, 5, -7, -9, -11, 13]",2,8,13,"[{'S': 6, 'ψ': [(3, -9), (3, 3), (5, -11), (-7, 13)]}]","[{'DG': 1, 'MG': 3}]",-4
4,"[0, -1, -2]","[-1, -6, -5, -6]","[1, -6, -6, -6, 7, 11, 18, -19]",2,8,19,"[{'S': 12, 'ψ': [(1, 11), (-6, -6), (7, -19), (-6, 18)]}]","[{'DG': 1, 'MG': 3}]",-4
5,"[-3, -4, -3]","[-5, 3, -7]","[3, 5, 5, 7, -9, -15, -15, 19]",2,8,19,"[{'S': 10, 'ψ': [(3, 7), (5, 5), (-9, 19), (5, -15)]}]","[{'DG': 2, 'MG': 2}]",-4
6,"[3, 0, -5]","[-1, 1, -1]","[4, 4, 4, 11, -12, -13, -19, 21]",8,8,21,"[{'S': 8, 'ψ': [(4, 4), (4, -12), (11, -19), (-13, 21)]}]","[{'DG': 1, 'MG': 3}]",-4
7,"[0, -2, 0]","[-3, -1, 1, -1]","[5, 5, 5, -15, 19, -21, -29, 31]",2,8,31,"[{'S': 10, 'ψ': [(5, 5), (19, -29), (-21, 31), (5, -15)]}]","[{'DG': 1, 'MG': 3}]",-4


In [44]:
hs[hs['n']==9][:3]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
1351,"[-2, 0, 1, 3]","[-1, 0, -1, 3]","[1, 1, 2, 2, -6, 9, -10, -12, 13]",1,9,13,"[{'S': 3, 'ψ': [(9, -12), (-6, 9), (1, 2), (-10, 13)]}]","[{'DG': 2, 'MG': 0}]",-2
1352,"[2, -2, -1]","[-2, -5, -3, -1]","[2, -3, 5, 8, -12, 13, -14, -23, 24]",2,9,24,"[{'S': 10, 'ψ': [(5, 5), (2, -12), (-14, 24), (-3, 13), (13, -23), (2, 8)]}]","[{'DG': 1, 'MG': 1}]",-2
1353,"[1, 2, 1]","[-1, -3, -2, -1]","[1, 6, 7, 8, 9, -15, -22, -23, 29]",1,9,29,"[{'S': 14, 'ψ': [(7, 7), (8, -22), (9, -23), (6, 8), (1, -15), (-15, 29)]}]","[{'DG': 1, 'MG': 1}]",-2


In [45]:
hs[hs['n']==11][:4]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
1366,"[-2, -1, -2, -1, 3]","[-3, -1, 2, -1, 3]","[1, 1, 1, 1, -3, 5, -6, 8, -9, -10, 11]",2,11,11,"[{'S': 2, 'ψ': [(-9, 11), (-6, 8), (1, 1), (8, -10), (1, -3), (-3, 5)]}]","[{'DG': 1, 'MG': 4}]",-5
1367,"[-4, 1, 0, 4]","[-1, -3, 1, 0, 2]","[1, 4, 4, 4, 4, -8, -9, 16, -17, -24, 25]",4,11,25,"[{'S': 8, 'ψ': [(4, 4), (-8, 16), (-17, 25), (1, -9), (16, -24)]}]","[{'DG': 1, 'MG': 4}]",-5
1368,"[-2, 1, 0, -1]","[-2, -4, -2, 1, 2]","[1, -2, -2, -4, -5, 6, 6, -7, 8, 11, -12]",1,11,12,"[{'S': 4, 'ψ': [(-7, 11), (-2, -2), (8, -12), (1, -5), (-2, 6), (-4, 8)]}]","[{'DG': 2, 'MG': 2}]",-4
1369,"[-3, 0, -3, 0, 2]","[-3, -2, 0, -1, 1]","[2, 2, 4, -6, -6, -8, 12, -15, 17, 19, -21]",4,11,21,"[{'S': 4, 'ψ': [(2, 2), (-8, 12), (2, -6), (-15, 19), (4, -8), (17, -21)]}]","[{'DG': 2, 'MG': 2}]",-4


In [46]:
hs[hs['n']==12][:4]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
1739,"[1, 0, -2, 0, -2]","[-3, -1, 1, -1, 1]","[1, 1, 7, 7, 7, 7, 7, -11, -15, -15, -21, 25]",2,12,25,"[{'S': 14, 'ψ': [(-11, 25), (7, -21), (1, -15), (7, 7)]}]","[{'DG': 2, 'MG': 5}]",-7
1740,"[-1, 0, -3, 0, -1]","[-1, 1, -1, -3, -1]","[5, 5, 7, 7, 7, 7, 7, -15, -19, -19, -21, 29]",2,12,29,"[{'S': 14, 'ψ': [(5, -19), (7, -21), (-15, 29), (7, 7)]}]","[{'DG': 2, 'MG': 5}]",-7
1741,"[2, 0, -1, 0, 1]","[-3, -2, -3, -2, -1]","[1, -2, -2, -3, 4, 4, 4, 4, -5, -6, -6, 7]",2,12,7,"[{'S': 8, 'ψ': [(4, 4), (1, 7), (-3, -5), (-2, -6)]}]","[{'DG': 2, 'MG': 4}]",-6
1742,"[-1, 1, -2, -1, 1]","[-2, 0, -1, -3, 0]","[1, 1, 1, 1, 1, 2, -3, -4, 5, -6, -7, 8]",1,12,8,"[{'S': 2, 'ψ': [(-6, 8), (1, 1), (1, -3), (2, -4), (5, -7), (-3, 5)]}]","[{'DG': 1, 'MG': 5}]",-6


In [47]:
hs3['Gmax']=-hs3['Generations'].str[0].apply(lambda d: sum(d.values()))
hs3=hs3.sort_values(['n','Gmax']).reset_index(drop=True)

In [48]:
hs3[hs3['n']==8][:1]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
0,"[1, 0, -2]","[-3, -1, 1]","[3, 3, 3, 5, -7, -9, -11, 13]",2,8,13,"[{'S': 18, 'ψ': [(-7, -11), (-9, -9), (5, 13)]}]","[{'DG': 1, 'MG': 1}]",-2


In [49]:
hs3[hs3['n']==9][:4]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
3,"[-2, -1, 0]","[-1, -3, -5, -3]","[5, 6, 6, 6, -8, -9, -9, -10, 13]",2,9,13,"[{'S': 18, 'ψ': [(-8, -10), (-9, -9), (5, 13)]}]","[{'DG': 1, 'MG': 2}]",-3
4,"[1, -2, -1]","[-4, -9, -5, -3]","[1, -2, 6, 6, 6, -9, -9, -16, 17]",6,9,17,"[{'S': 18, 'ψ': [(1, 17), (-2, -16), (-9, -9)]}]","[{'DG': 1, 'MG': 2}]",-3
5,"[-2, 0, 2]","[-1, 1, 0, -1]","[1, 1, -4, -5, 9, 9, 9, -10, -10]",1,9,10,"[{'S': 9, 'ψ': [(1, -10), (-4, -5)]}]","[{'DG': 2, 'MG': 0}]",-2
6,"[2, -2, 1, 3]","[-5, -4, -3, -1]","[1, -2, -2, -2, 5, -7, 8, 9, -10]",1,9,10,"[{'S': 2, 'ψ': [(8, -10), (5, -7), (1, 1), (-7, 9)]}]","[{'DG': 1, 'MG': 1}]",-2


In [50]:
hs3[hs3['n']==10][:5]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
44,"[-4, -1, 0, 1]","[-4, 1, -1, -3, 1]","[1, 1, 1, 2, 2, -7, 11, -12, -15, 16]",3,10,16,"[{'S': 4, 'ψ': [(-7, 11), (11, -15), (-12, 16), (2, 2)]}]","[{'DG': 1, 'MG': 2}]",-3
45,"[-3, 3, 0, -6]","[-3, -2, 3, 4]","[3, 3, 3, -4, 9, 9, -10, -14, -27, 28]",18,10,28,"[{'S': 18, 'ψ': [(-10, 28), (9, 9), (9, -27), (-4, -14)]}]","[{'DG': 1, 'MG': 2}]",-3
46,"[-1, 0, -4, -3]","[-3, 1, -1, -5]","[1, -3, -3, -3, -9, -9, 11, 17, 27, -29]",2,10,29,"[{'S': 18, 'ψ': [(1, 17), (-9, 27), (11, -29), (-9, -9)]}]","[{'DG': 1, 'MG': 2}]",-3
47,"[0, 1, 0, -2]","[-1, -3, 3, -1, -3]","[1, -5, -7, 9, -11, -13, 15, 15, 15, -19]",4,10,19,"[{'S': 18, 'ψ': [(-7, -11), (-5, -13), (1, -19), (9, 9)]}]","[{'DG': 1, 'MG': 1}]",-2
48,"[1, 0, 2, 0]","[-3, 0, -3, -2, -1]","[1, -2, 3, 3, 3, -6, 9, -12, -19, 20]",6,10,20,"[{'S': 18, 'ψ': [(-6, -12), (-2, 20), (1, -19), (9, 9)]}]","[{'DG': 1, 'MG': 1}]",-2


In [51]:
hs3[hs3['n']==11][:2]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
65,"[2, -1, 2, -1]","[-2, -1, -2, 2, 4]","[4, 5, 8, -9, -9, 10, -12, -12, -12, 13, 14]",1,11,14,"[{'S': 18, 'ψ': [(8, 10), (-9, -9), (5, 13), (4, 14)]}]","[{'DG': 1, 'MG': 2}]",-3
66,"[2, 3, 0, -3, 3]","[0, -3, 1, -1, -3]","[1, 1, -2, -3, 6, 6, 6, -7, -10, -10, 12]",15,11,12,"[{'S': 9, 'ψ': [(1, -10), (-3, 12), (-2, -7)]}]","[{'DG': 2, 'MG': 0}]",-2


In [52]:
hs3[hs3['n']==12][:5]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
84,"[1, 0, 2, 0, 2]","[-3, -1, -3, -1, -3]","[7, 9, 9, 9, 11, -13, -13, -27, -27, -27, 31, 31]",2,12,31,"[{'S': 18, 'ψ': [(7, 11), (-13, 31), (9, 9)]}]","[{'DG': 2, 'MG': 3}]",-5
85,"[-3, -1, -2, 3, -3]","[-3, -2, -3, -2, 0]","[3, 3, 3, 6, 7, -8, -9, -9, -9, -10, 11, 12]",19,12,12,"[{'S': 18, 'ψ': [(6, 12), (-8, -10), (7, 11), (-9, -9)]}]","[{'DG': 1, 'MG': 3}]",-4
86,"[-3, -1, 2, 0, -3]","[-3, -2, -1, -2, -3]","[3, 4, -8, -9, -9, -9, -10, 14, 15, 15, 15, -21]",12,12,21,"[{'S': 18, 'ψ': [(3, -21), (-9, -9), (-8, -10), (4, 14)]}]","[{'DG': 1, 'MG': 3}]",-4
87,"[-3, 0, -3, -1, 3]","[-3, -1, -3, -1, 1]","[2, 6, 6, 6, -7, -16, -20, -20, -20, 21, 21, 21]",8,12,21,"[{'S': 14, 'ψ': [(6, -20), (2, -16), (-7, -7)]}]","[{'DG': 3, 'MG': 1}]",-4
88,"[1, 0, -3, 0, -2]","[-3, -1, -3, -1, -3]","[3, -5, -7, -7, -7, -9, 15, -17, 21, 21, 21, -29]",4,12,29,"[{'S': 14, 'ψ': [(-7, -7), (15, -29), (-5, -9), (3, -17)]}]","[{'DG': 1, 'MG': 3}]",-4
